<code>
Model	MAE	MSE	RMSE	R2	RMSLE	MAPE	TT (Sec)
<br>huber	Huber Regressor	176.5214	87177.3210	295.2018	0.9995	0.0076	0.0048	0.2767
<br>omp	Orthogonal Matching Pursuit	176.8994	87074.0508	295.0281	0.9995	0.0076	0.0048	0.2200
<br>llar	Lasso Least Angle Regression	176.9285	87099.6957	295.0704	0.9995	0.0076	0.0048	0.2167
<br>lasso	Lasso Regression	176.9336	87099.0767	295.0690	0.9995	0.0076	0.0048	0.3600
<br>lr	Linear Regression	176.9777	87128.9787	295.1192	0.9995	0.0076	0.0048	0.3633
<br>lar	Least Angle Regression	176.9777	87128.9787	295.1192	0.9995	0.0076	0.0048	0.4300
<br>br	Bayesian Ridge	176.9778	87128.8999	295.1190	0.9995	0.0076	0.0048	0.2533
<br>ridge	Ridge Regression	177.0186	87108.0300	295.0839	0.9995	0.0076	0.0048	0.3833
<br>par	Passive Aggressive Regressor	177.6973	87356.2870	295.5045	0.9995	0.0076	0.0048	0.2000
<br>lightgbm	Light Gradient Boosting Machine	195.5358	99140.7557	314.6561	0.9995	0.0081	0.0053	0.9300
<br>rf	Random Forest Regressor	199.4361	101335.1889	318.2649	0.9994	0.0083	0.0054	5.1033
<br>gbr	Gradient Boosting Regressor	201.3509	99449.4057	315.2657	0.9995	0.0083	0.0056	1.7800
<br>et	Extra Trees Regressor	201.5587	105765.2161	325.1893	0.9994	0.0084	0.0054	2.1867
<br>xgboost	Extreme Gradient Boosting	210.7075	108984.0729	329.9108	0.9994	0.0086	0.0058	1.3700
<br>dt	Decision Tree Regressor	263.7993	175515.6021	418.9104	0.9990	0.0109	0.0072	0.2100
<br>catboost	CatBoost Regressor	364.3085	253714.2094	503.3081	0.9986	0.0134	0.0103	17.0400
<br>ada	AdaBoost Regressor	615.4694	637822.1677	798.6293	0.9965	0.0349	0.0231	0.8600
<br>knn	K Neighbors Regressor	772.0608	1201267.8542	1074.7690	0.9934	0.0333	0.0234	0.2367
<br>en	Elastic Net	3640.4392	18414889.5698	4291.1610	0.8990	0.1410	0.1205	0.3700
<br>dummy	Dummy Regressor	11623.2256	182392298.6667	13505.1523	-0.0005	0.4012	0.3887	0.2167
</code>

In [ ]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/crypto'))
print(sys.path)

In [ ]:
import pandas as pd
from pycaret.regression import *
from src.utils import *
from src.calcEMA import calc_RSI

In [ ]:
# Variables
datadir = './data'
label = 'close'

### Metadata

<code>
Field Name - Description</br>
open_time - Kline Open time in unix time format</br>
open - Open Price</br>
high - High Price</br>
low	- Low Price</br>
close	- Close Price</br>
volume - Volume</br>
close_time - Kline Close time in unix time format</br>
quote_volume - Quote Asset Volume</br>
count	- Number of Trades</br>
taker_buy_volume - Taker buy base asset volume during this period</br>
taker_buy_quote_volume - Taker buy quote asset volume during this period</br>
ignore - Ignore</br>
</code>

In [ ]:
print(use_cols)
data = read_data(datadir)
data.head()

In [ ]:
data = calc_RSI(data, label)
data.dropna(inplace=True)
print(data.info())
data.head()

In [ ]:
data.tail(1)

In [ ]:
df_predict = forecast(data, 48, fold=10)
df_predict.columns = df_predict.columns + '_p'

In [ ]:
df_predict

In [ ]:
test_dir = './test'
test_data = read_data(test_dir)
#test_data = calc_RSI(test_data, label)
test_data.dropna(inplace=True)

print(test_data.info())
test_data

In [ ]:
import plotly.express as px

start_date = df_predict["open_time_p"].min()#strftime("%Y-%m-%d")
end_date = df_predict["open_time_p"].max()#.strftime("%Y-%m-%d")
#now = datetime.now().strftime("%Y-%m-%d")

df_predict.index = df_predict['open_time_p']
test_data.index = test_data['open_time']

filtered_data = test_data.loc[(test_data['open_time'] >= start_date) & (test_data['open_time'] <= end_date)]
filtered_data = pd.concat([filtered_data, df_predict], axis=1)
filtered_data.drop(columns=['open_time', 'open_time_p'], inplace=True)
filtered_data.round(2)

fig1 = px.line(
    filtered_data, x=filtered_data.index, y=['close', 'close_p'], template = 'plotly_dark', 
    range_x=[start_date, end_date])
fig1.show()